DATASET LOADING FROM KAGGLE

In [27]:
!pip install opendatasets

In [28]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/shrutimechlearn/churn-modelling/data")

Skipping, found downloaded files in "./churn-modelling" (use force=True to force download)


In [29]:
import numpy as np
import pandas as pd

In [30]:
df = pd.read_csv("/content/churn-modelling/Churn_Modelling.csv")
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [31]:
df1 = df.drop(columns =['RowNumber','CustomerId','Surname',])
df1

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


EDA

In [32]:
!pip install ydata-profiling

In [33]:
from ydata_profiling import ProfileReport
prof = ProfileReport(df1)
prof.to_file(output_file='hitesh.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 11/11 [00:00<00:00, 33.10it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

FEATURE ENGINEERING

In [34]:
df1.sample(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
6764,570,France,Male,37,6,0.00,1,1,1,187758.50,0
8250,850,France,Female,35,9,0.00,2,0,0,25329.48,0
3775,528,Germany,Male,22,5,93547.23,2,0,1,961.57,0
7383,737,Spain,Female,39,7,130051.66,2,0,0,55356.39,1
9244,546,Germany,Female,25,3,132837.70,1,1,0,131647.31,0


In [35]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline , make_pipeline


In [36]:
X_train , X_test , y_train , y_test = train_test_split(df1.drop(columns = ['Exited'],axis =1),df1['Exited'], test_size = 0.3, random_state = 0)

In [37]:
X_train

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
7681,641,France,Male,33,2,146193.60,2,1,1,55796.83
9031,541,France,Male,39,7,0.00,2,1,0,19823.02
3691,590,France,Female,76,5,160979.68,1,0,1,13848.58
202,516,Spain,Male,50,5,0.00,1,0,1,146145.93
5625,508,France,Female,60,7,143262.04,1,1,1,129562.74
...,...,...,...,...,...,...,...,...,...,...
9225,594,Germany,Female,32,4,120074.97,2,1,1,162961.79
4859,794,Spain,Female,22,4,114440.24,1,1,1,107753.07
3264,738,France,Male,35,5,161274.05,2,1,0,181429.87
9845,590,Spain,Female,38,9,0.00,2,1,1,148750.16


In [38]:
y_train

,Exited
7681,1
9031,0
3691,0
202,1
5625,0
...,...
9225,0
4859,0
3264,0
9845,0


In [39]:
#onehotencoding
tnf1 = ColumnTransformer([('one_hot_encode',OneHotEncoder(sparse_output =False , drop = 'first',handle_unknown='ignore'),[1,2])])

In [40]:
#feature scaling
tnf2 = ColumnTransformer([('feature_scaling', MinMaxScaler(),slice(0,10))])

In [41]:
#feature selection
tnf3 = SelectKBest(score_func=chi2,k=6)

In [42]:
#train the model
tnf4 = LogisticRegression()

PIPELINE


In [43]:
pipe = make_pipeline(tnf1,tnf2,tnf3,tnf4)

In [44]:
pipe.fit(X_train,y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:783: UserWarning: k=6 is greater than n_features=3. All the features will be returned.
  warnings.warn(


Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(transformers=[('one_hot_encode',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 2])])),
                ('columntransformer-2',
                 ColumnTransformer(transformers=[('feature_scaling',
                                                  MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=6,
                             score_func=<function chi2 at 0x7d98510b1760>)),
                ('logisticregression', LogisticRegression())])

PREDICTION AND ACCURACY

In [45]:
y_pred = pipe.predict(X_test)

In [46]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [47]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.793

EXPORTING THE PIPELINE

In [48]:
import pickle

In [49]:
pickle.dump(pipe,open('pipe.pkl','wb'))

TESTING


In [50]:
import pickle

try:
    with open('pipe.pkl', 'rb') as f:
        loaded_pipe = pickle.load(f)
    print("Pipeline loaded successfully!")
    # You can now use 'loaded_pipe'
    # For example, to check its type or content:
    # print(type(loaded_pipe))
except EOFError:
    print("Error: The 'pipe.pkl' file is empty or corrupted. Please ensure it was created correctly by re-running the 'pickle.dump' cell with a 'with' statement.")
except Exception as e:
    print(f"An unexpected error occurred while loading the pipeline: {e}")


Pipeline loaded successfully!


In [51]:
random_data = np.array([400,'Spain','Female',40,5,10000,1,1,1,25000],dtype =object).reshape(1,10)

In [52]:
pipe.predict(random_data)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


array([0])